# 快乐屋的人脸识别

本节，你将构建一个人脸识别系统。许多这里介绍的思想来自 [FaceNet](https://arxiv.org/pdf/1503.03832.pdf)。在课程中，我们还讨论了 [DeepFace](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf)。

人脸识别问题通常分为两类：

- **人脸验证 (Face Verification)** - “这个人是否是声称的那个人？”。例如，在一些机场，你可以通过让系统扫描护照并验证你（持有护照的人）是否为正确身份来通过海关。用人脸解锁的手机也在使用人脸验证。这是一个 **1:1 匹配问题**。
- **人脸识别 (Face Recognition)** - “这个人是谁？”。例如，视频课程展示了一个人脸识别视频 (https://www.youtube.com/watch?v=wr4rx0Spihs)，百度员工可以不用额外身份验证就进入办公室。这是一个 **1:K 匹配问题**。

本节使用facenet_pytorch，可以将一张人脸图像编码为一个512维向量。通过比较两个这样的向量，就能判断两张图片是否是同一个人。

**本节中，你将：**
- 实现三元组损失函数 (triplet loss function)
- 使用预训练模型将人脸图像映射到512 维编码
- 使用这些编码来进行人脸验证和人脸识别

在本节中，我们将使用一个预训练模型，该模型采用 **“channels first”** 的卷积网络激活表示方式，而不是之前中使用的 **“channels last”** 方式。换句话说，一批图像的形状是 $(m, n_C, n_H, n_W)$ 而不是 $(m, n_H, n_W, n_C)$。在开源实现中，这两种方式都有一定应用；深度学习社区中还没有统一标准。

让我们加载所需的包吧。


## 1 - 导入库

In [1]:
# ===============================================================
# ========== 导入必要的库 ==========
# ===============================================================

# -----------------------------
# PyTorch 核心库
# -----------------------------
import torch                    # PyTorch 主库，用于进行张量（Tensor）运算和自动微分（autograd）
import torch.nn as nn           # nn 模块，包含神经网络的常用层（如卷积层、线性层等）
import torch.nn.functional as F # F 模块（functional API），提供函数式操作（如 ReLU、池化、卷积等）
# 💡 区别说明：
# - nn.Module：定义层结构（类的方式）
# - torch.nn.functional：直接调用函数（函数式调用）

# -----------------------------
# 图像处理库
# -----------------------------
from torchvision import transforms  # torchvision.transforms 提供常见图像预处理功能，如Resize、ToTensor、Normalize等
from PIL import Image              # PIL（Python Imaging Library）用于加载、显示和保存图片
import cv2                         # OpenCV库，用于高级图像处理（如滤波、边缘检测等）

# -----------------------------
# 数据处理库
# -----------------------------
import os                          # OS模块，用于路径拼接、文件读取、文件夹操作等
import numpy as np                 # NumPy 是科学计算核心库，用于矩阵运算、线性代数、随机数生成等
from numpy import genfromtxt       # 从文本文件中读取结构化数据（例如csv）
import pandas as pd                # Pandas 用于表格数据（DataFrame）的读取、处理与分析
import matplotlib.pyplot as plt    # Matplotlib 的 pyplot 接口，用于绘制图像（如损失曲线、结果对比等）

# -----------------------------
# 预训练模型
# -----------------------------
from facenet_pytorch import InceptionResnetV1

# ===============================================================
# ========== Jupyter Notebook 配置 ==========
# ===============================================================

# 在 Notebook 中内联显示图像（绘图结果会直接显示在单元格中）
%matplotlib inline  

# 自动重新加载模块（使得修改的 Python 文件可立即生效）
%load_ext autoreload

# 自动重新加载所有模块（除非明确排除），相当于动态加载
%autoreload 2

# - 这几行命令仅在 Jupyter Notebook 环境中有效
# - 主要用于加快实验调试流程，无需每次手动重启内核即可加载修改后的代码

# ===============================================================
# ========== 设备配置 ==========
# ===============================================================

# torch.device 用于指定计算设备：
# - 若电脑支持 CUDA 且驱动正确安装，则优先使用 GPU；
# - 否则自动回退到 CPU。
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 打印当前使用的设备（cuda 或 cpu）
print(f"Using device: {device}")

# 提示：
# - 使用 GPU 可以显著提升深度学习模型训练速度；
# - 若输出为 "cpu"，说明未检测到可用 GPU；
# - 可以使用 torch.cuda.get_device_name(0) 查看 GPU 型号。


Using device: cpu


## 2 - 朴素的人脸验证 (Naive Face Verification)

在人脸验证 (Face Verification) 中，你会得到两张图像，并需要判断它们是否属于同一个人。最简单的方法是逐像素比较两张图像。如果两张原始图像之间的距离小于某个选定的阈值，那么它们可能是同一个人！

<img src="images/pixel_comparison.png" style="width:380px;height:150px;">
<caption><center> <u> <font color='purple'> ** 图 1 ** </u></center></caption>


当然，这种算法的效果非常差，因为像素值会因光照、人物脸部的朝向、甚至头部位置的细微变化而发生剧烈改变。  

你将会看到，与其直接使用原始图像，不如学习一个编码函数 $f(img)$。通过对这种编码进行逐元素比较，可以更准确地判断两张图片是否是同一个人。  


## 3 - 将人脸图像编码为512维向量

### 3.1 - 使用卷积神经网络计算编码


FaceNet 是通过最小化三元组损失来训练的。但由于训练需要大量数据和计算资源，我们这里不从零开始训练。  
相反，我们将加载一个已经训练好的模型。使用下面的单元格来加载模型。

In [2]:
# ========================================
# 加载模型与定义编码函数
# ========================================

# ========== 使用预训练模型 ==========
try:
    # 打印分割线，美化输出
    print("\n" + "=" * 70)
    print("加载预训练 FaceNet 模型")
    print("=" * 70)

    # ------------------------------------------------------------
    # 创建 FaceNet 模型实例
    # ------------------------------------------------------------
    # InceptionResnetV1 是 facenet_pytorch 提供的 Inception-ResNet v1 架构
    # 参数解释：
    # - pretrained='vggface2' ：加载在 VGGFace2 数据集上预训练的权重
    # - classify=False ：不进行分类任务，仅输出特征向量
    # - device=device ：指定运行设备，可为 'cpu' 或 'cuda'
    FRmodel = InceptionResnetV1(
        pretrained='vggface2',  # 选择预训练权重
        classify=False,          # 仅提取人脸特征向量，不进行分类
        device=device            # 模型运行设备
    ).eval()  # eval() 将模型设置为推理模式，关闭 dropout 和 BatchNorm 更新

    # 打印模型加载成功信息
    print(f"\n✓ 预训练模型加载成功！")
    print(f"  - 模型: InceptionResnetV1")
    print(f"  - 预训练数据集: VGGFace2")
    print(f"  - 编码维度: 512")  # 输出特征向量的维度
    print(f"  - 设备: {device}")

    # ------------------------------------------------------------
    # 定义图像编码函数：将图像转换为 512 维人脸特征向量
    # ------------------------------------------------------------
    def img_to_encoding(image_path, model):
        """
        使用预训练 FaceNet 模型提取人脸编码（512维特征向量）

        参数:
            image_path: 图像路径（str）
            model: 已加载的 InceptionResnetV1 模型对象

        返回:
            embedding: numpy 数组形式的 512 维人脸特征向量
        """

        # 打开图像并转换为 RGB 模式，确保通道为 3
        img = Image.open(image_path).convert('RGB')

        # FaceNet 模型默认输入大小为 160x160
        img = img.resize((160, 160))

        # 将图像转换为 numpy 数组 (H, W, C)
        img_array = np.array(img)

        # --------------------------------------------------------
        # 转换为 PyTorch 张量
        # --------------------------------------------------------
        # torch.from_numpy(img_array) ：将 numpy 数组转为 torch 张量
        # permute(2, 0, 1) ：将通道顺序从 (H, W, C) 转换为 (C, H, W)
        # float() ：转换为浮点类型，便于后续归一化和模型输入
        img_tensor = torch.from_numpy(img_array).permute(2, 0, 1).float()

        # --------------------------------------------------------
        # 归一化到 [-1, 1]，与模型训练时一致
        # 公式: (像素值 - 127.5) / 128
        # 像素值原范围 [0, 255]
        # --------------------------------------------------------
        img_tensor = (img_tensor - 127.5) / 128.0

        # 增加 batch 维度，模型输入形状为 (1, 3, 160, 160)
        img_tensor = img_tensor.unsqueeze(0).to(device)  # 送入指定设备

        # --------------------------------------------------------
        # 前向传播提取特征向量
        # --------------------------------------------------------
        model.eval()  # 确保模型为推理模式
        with torch.no_grad():  # 关闭梯度计算，提高推理速度，减少显存占用
            embedding = model(img_tensor)  # 输出形状: (1, 512)

        # 转为 numpy 数组并返回
        return embedding.cpu().numpy()  # cpu() 转回 CPU，方便后续 numpy 操作

    # ------------------------------------------------------------
    # 设置人脸识别阈值
    # ------------------------------------------------------------
    THRESHOLD = 0.8  # 512 维向量相似度阈值，通常在 0.7~1.0
    ENCODING_DIM = 512  # 人脸特征向量维度

    print(f"\n✓ 编码函数已配置")
    print(f"  - 图像尺寸: 160x160")
    print(f"  - 归一化: [-1, 1]")
    print(f"  - 识别阈值: {THRESHOLD}")
    print("=" * 70)

# ------------------------------------------------------------
# 如果未安装 facenet-pytorch，则提示安装
# ------------------------------------------------------------
except ImportError:
    print("\n" + "=" * 70)
    print("❌ 错误: 未安装 facenet-pytorch")
    print("=" * 70)
    print("\n请运行以下命令安装:")
    print("  pip install facenet-pytorch")
    print("\n或者修改配置使用随机权重模型:")
    print("  USE_PRETRAINED = False")
    print("=" * 70)
    raise  # 抛出异常终止程序



加载预训练 FaceNet 模型

✓ 预训练模型加载成功！
  - 模型: InceptionResnetV1
  - 预训练数据集: VGGFace2
  - 编码维度: 512
  - 设备: cpu

✓ 编码函数已配置
  - 图像尺寸: 160x160
  - 归一化: [-1, 1]
  - 识别阈值: 0.8


你需要了解的关键点：

- 该网络以 160x160 的 RGB 图像作为输入。具体来说，它将一张人脸图像（或一个包含 $m$ 张人脸图像的批次）作为张量输入，形状为 $(m, n_C, n_H, n_W) = (m, 3, 160, 160)$
- 输出为一个形状为 $(m, 512)$ 的矩阵，将每张输入的人脸图像编码为一个 512 维的向量



In [3]:
# =========================================
# 计算模型参数总数
# =========================================

# sum(p.numel() for p in FRmodel.parameters())
# 解释：
# - FRmodel.parameters() ：
#       获取模型 FRmodel 中所有可训练和非可训练的参数（weights 和 biases）。
#       返回一个生成器，每个元素是一个张量（torch.Tensor）。
# - p.numel() ：
#       返回张量 p 中元素的总数，即该参数张量包含多少个标量。
#       例如，一个形状为 [3, 3, 64, 128] 的卷积核张量，p.numel() = 3*3*64*128
# - for p in FRmodel.parameters() ：
#       遍历模型的每一个参数张量。
# - sum(...) ：
#       将所有参数张量的元素数量累加，得到模型总参数量。

total_params = sum(p.numel() for p in FRmodel.parameters())  # 计算并保存模型总参数数目

# 打印模型总参数数目
print("Total Params:", total_params)  
# 输出示例：Total Params: 2181234
# 其中 2181234 表示模型中所有参数的总数


Total Params: 27910327


通过使用一个具有 512 个神经元的全连接层作为最后一层，模型保证输出是一个 512 维的编码向量（图中的128-d需修正为512-d）。你可以使用这些编码来比较两张人脸图像，如下图所示：

<img src="images/distance_kiank.png" style="width:680px;height:250px;">
<caption><center> <u> <font color='purple'> **图 2**: <br> </u> <font color='purple'> 通过计算两个编码之间的距离并设定阈值，你可以判断这两张图片是否属于同一个人 </center></caption>

因此，一个好的编码应满足：
- 同一个人的两张图像的编码应该彼此非常相似
- 不同人的两张图像的编码应该相差很大

三元组损失函数（triplet loss）将这一思想形式化，它试图“拉近”同一人的两张图像（Anchor 和 Positive）的编码，同时“推远”不同人的两张图像（Anchor 和 Negative）的编码。

<img src="images/triplet_comparison.png" style="width:280px;height:150px;">
<br>
<caption><center> <u> <font color='purple'> **图 3**: <br> </u> <font color='purple'> 在接下来的部分中，我们将从左到右称这三张图像为：Anchor (A), Positive (P), Negative (N)  </center></caption>


### 3.2 - 三元组损失（Triplet Loss）

对于一张图像 $x$，记其编码为 $f(x)$，其中 $f$ 是神经网络计算的函数。

<img src="images/f_x.png" style="width:380px;height:150px;">

训练时使用三元组图像 $(A, P, N)$：  

- **A（Anchor）**：基准图像，即某个人的照片。
- **P（Positive）**：正样本图像，与 Anchor 是同一个人。
- **N（Negative）**：负样本图像，与 Anchor 是不同的人。

这些三元组从训练数据集中选取。用 $(A^{(i)}, P^{(i)}, N^{(i)})$ 表示第 $i$ 个训练样本。  

我们希望确保图像 $A^{(i)}$ 与正样本 $P^{(i)}$ 的距离比与负样本 $N^{(i)}$ 的距离小，并且至少比负样本距离小一个 margin $\alpha$：

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

因此，我们希望最小化以下“三元组损失”：

$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

这里，$[z]_+$ 表示 $max(z,0)$。  

说明：
- (1) 项是 Anchor 与 Positive 的平方距离，你希望它尽可能小。
- (2) 项是 Anchor 与 Negative 的平方距离，你希望它尽可能大，因此前面加了负号。
- $\alpha$ 是 margin，是一个需要手动设置的超参数。本作业中使用 $\alpha = 0.2$。

大部分实现还会对编码向量进行归一化，使其范数为 1（即 $\mid \mid f(img)\mid \mid_2 = 1$）；这里不需要考虑。

**说明**：实现公式 (3) 定义的三元组损失。步骤如下：
1. 计算 Anchor 与 Positive 编码的距离：$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2$
2. 计算 Anchor 与 Negative 编码的距离：$\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$
3. 对每个训练样本计算公式：$ \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2 + \alpha$
4. 对所有训练样本取最大值与 0，并求和，得到总损失：
$$\mathcal{J} = \sum^{m}_{i=1} \large[ \small \mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 - \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2 + \alpha \large ] \small_+ \tag{4}$$

可用函数：`torch.sum()`, `torch.square()`, `torch.maximum()`。  
步骤 1 和 2 需要对向量的各个分量求和（使用 `dim=-1`），步骤 4 需要对训练样本求和。


In [4]:
# ========================================
# 三元组损失函数 (Triplet Loss)
# ========================================

def triplet_loss(y_true, y_pred, alpha=0.2):
    """
    实现三元组损失函数，用于训练人脸识别模型
    
    三元组损失的目标:
    - 使同一个人的图像(Anchor和Positive)在特征空间中更接近
    - 使不同人的图像(Anchor和Negative)在特征空间中更远离
    
    数学公式:
        L = Σ max(||f(A) - f(P)||² - ||f(A) - f(N)||² + α, 0)
    
    其中:
    - f(A): Anchor图像的编码
    - f(P): Positive图像的编码(与Anchor是同一人)
    - f(N): Negative图像的编码(与Anchor是不同人)
    - α: margin参数，确保正负样本之间有足够的分离度
    
    参数:
        y_true: 真实标签(在此函数中不使用，为了兼容Keras接口)
        y_pred (tuple): 包含3个tensor的元组
            - anchor (torch.Tensor): Anchor图像编码，形状(batch_size, 512)
            - positive (torch.Tensor): Positive图像编码，形状(batch_size, 512)
            - negative (torch.Tensor): Negative图像编码，形状(batch_size, 512)
        alpha (float): margin参数，默认为0.2
            - 表示正负样本之间的最小期望距离差
            - 值越大表示要求正负样本分离得越远
    
    返回:
        loss (torch.Tensor): 标量，三元组损失值
            - 如果为0，表示模型已经很好地分离了正负样本
            - 如果>0，表示需要继续训练增大正负样本距离差
    """
    
    # -----------------------------
    # Step 0: 解包三元组编码
    # -----------------------------
    # y_pred[0]: Anchor编码
    # y_pred[1]: Positive编码
    # y_pred[2]: Negative编码
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    # -----------------------------
    # Step 1: 计算Anchor与Positive之间的距离平方
    # -----------------------------
    # torch.square(x) : 对每个元素平方 (x^2)
    # anchor - positive : element-wise 差
    # torch.sum(..., dim=-1) : 对最后一维求和，得到L2距离的平方
    # 输出形状: (batch_size,)
    pos_dist = torch.sum(torch.square(anchor - positive), dim=-1)
    
    # -----------------------------
    # Step 2: 计算Anchor与Negative之间的距离平方
    # -----------------------------
    # 与上一步类似
    neg_dist = torch.sum(torch.square(anchor - negative), dim=-1)
    
    # -----------------------------
    # Step 3: 计算基础损失
    # -----------------------------
    # 基础损失公式: pos_dist - neg_dist + alpha
    # 含义:
    # - 如果pos_dist很小且neg_dist很大 → 基础损失为负 → 好情况
    # - 如果pos_dist很大或neg_dist很小 → 基础损失为正 → 需要优化
    # - alpha提供margin，使得正负样本差至少为alpha
    basic_loss = pos_dist - neg_dist + alpha
    
    # -----------------------------
    # Step 4: 应用ReLU并求和
    # -----------------------------
    # torch.maximum(basic_loss, 0.0) ：将负值裁剪为0，相当于 ReLU
    # torch.sum(...) ：对整个batch求和，得到标量损失
    loss = torch.sum(torch.maximum(basic_loss, torch.tensor(0.0)))
   
    # 返回损失值
    return loss


In [5]:
# ===============================================================
# ========== 测试 Triplet Loss 函数（适配 512 维 FaceNet 输出） ==========
# ===============================================================

print("=" * 60)
print("测试 Triplet Loss 函数 (FaceNet 512维)")
print("=" * 60)

# -----------------------------
# 测试1：随机向量
# -----------------------------
torch.manual_seed(1)
y_true = (None, None, None)
# anchor, positive, negative 维度均为 (batch_size, 512)
y_pred = (
    torch.normal(mean=6, std=0.1, size=(3, 512)),
    torch.normal(mean=1, std=1, size=(3, 512)),
    torch.normal(mean=3, std=4, size=(3, 512))
)
loss = triplet_loss(y_true, y_pred)
print(f"\n测试1 - 随机向量: loss = {loss.item():.4f}")
if loss.item() == 0:
    print("  注意：loss=0 说明 negative 距离已经足够远，这是正常的！")

# -----------------------------
# 测试2：设计的测试用例（positive和negative都很近）
# -----------------------------
print("\n测试2 - 设计用例（positive和negative都很近）:")
anchor = torch.zeros(3, 512)
positive = torch.ones(3, 512) * 0.1   # 距离anchor很近
negative = torch.ones(3, 512) * 0.15  # 距离anchor也很近，违反margin
y_pred2 = (anchor, positive, negative)
loss2 = triplet_loss(None, y_pred2)
print(f"  loss = {loss2.item():.4f}")

# -----------------------------
# 测试3：详细分析单样本
# -----------------------------
print("\n测试3 - 单样本距离详细分析:")
anchor = torch.tensor([[1.0] * 512])
positive = torch.tensor([[1.1] * 512])
negative = torch.tensor([[0.5] * 512])

pos_dist = torch.sum(torch.square(anchor - positive), dim=-1)
neg_dist = torch.sum(torch.square(anchor - negative), dim=-1)
basic_loss = pos_dist - neg_dist + 0.2

print(f"  Anchor-Positive 平方距离: {pos_dist.item():.4f}")
print(f"  Anchor-Negative 平方距离: {neg_dist.item():.4f}")
print(f"  Basic loss (pos - neg + alpha): {basic_loss.item():.4f}")

loss3 = triplet_loss(None, (anchor, positive, negative))
print(f"  最终 loss (max(basic_loss, 0)): {loss3.item():.4f}")

# -----------------------------
# 测试总结
# -----------------------------
print("\n" + "=" * 60)
print("✓ Triplet Loss 函数测试完成")
print("说明：loss=0 时表示模型已经很好地区分了正负样本")
print("=" * 60)


测试 Triplet Loss 函数 (FaceNet 512维)

测试1 - 随机向量: loss = 3880.9407

测试2 - 设计用例（positive和negative都很近）:
  loss = 0.0000

测试3 - 单样本距离详细分析:
  Anchor-Positive 平方距离: 5.1200
  Anchor-Negative 平方距离: 128.0000
  Basic loss (pos - neg + alpha): -122.6800
  最终 loss (max(basic_loss, 0)): 0.0000

✓ Triplet Loss 函数测试完成
说明：loss=0 时表示模型已经很好地区分了正负样本


这里展示了三个人脸编码之间的距离示例：

<img src="images/distance_matrix.png" style="width:380px;height:200px;">
<br>
<caption><center> <u> <font color='purple'> **图4**：</u> <br>  <font color='purple'> 三个人的编码之间距离示例</center></caption>

现在让我们使用这个模型来执行人脸验证和人脸识别！


## 4 - 应用模型


回到幸福屋！自从你在之前的程序中实现了房屋的幸福识别功能后，居民们一直过得很快乐。

然而，出现了一些问题：幸福屋变得太快乐了，以至于邻居里所有快乐的人都跑来你的客厅玩耍。房间变得非常拥挤，这对房屋居民产生了负面影响。而且，这些随机的快乐人还吃光了你所有的食物。

因此，你决定改变门禁政策，不再允许随机快乐的人进屋，即使他们很快乐！  
相反，你想建立一个 **人脸验证（Face Verification）** 系统，只允许列表上的指定人员进入。为了获得许可，每个人都必须刷身份证（ID 卡）在门口进行身份识别。人脸识别系统随后会检查他们是否确实是所声称的人。


### 4.1 - 人脸验证（Face Verification）

现在我们来建立一个数据库，每个允许进入幸福屋的人对应一个人脸编码向量。为了生成编码，我们使用 `img_to_encoding(image_path, model)`，它会将指定图像输入到模型中进行前向传播，从而得到该图像的人脸编码。

运行以下代码即可建立数据库（用 Python 字典表示）。这个数据库将每个人的姓名映射到其人脸的 128 维编码向量。


In [6]:
# ========================================
# 创建人脸编码数据库
# ========================================

# 创建一个空字典，用于存储每个授权人员的姓名和对应的人脸编码
# 数据结构: { '姓名': 512维人脸特征向量 }
database = {}

# 使用之前定义的 img_to_encoding 函数，将每个人的图像转换为 512 维特征向量
# 并存入数据库字典中
database["danielle"]    = img_to_encoding("images/danielle.png", FRmodel)  # Danielle 的编码
database["younes"]      = img_to_encoding("images/younes.jpg", FRmodel)    # Younes 的编码
database["tian"]        = img_to_encoding("images/tian.jpg", FRmodel)      # Tian 的编码
database["andrew"]      = img_to_encoding("images/andrew.jpg", FRmodel)    # Andrew 的编码
database["kian"]        = img_to_encoding("images/kian.jpg", FRmodel)      # Kian 的编码
database["dan"]         = img_to_encoding("images/dan.jpg", FRmodel)       # Dan 的编码
database["sebastiano"]  = img_to_encoding("images/sebastiano.jpg", FRmodel) # Sebastiano 的编码
database["bertrand"]    = img_to_encoding("images/bertrand.jpg", FRmodel)  # Bertrand 的编码
database["kevin"]       = img_to_encoding("images/kevin(1).jpg", FRmodel)  # Kevin 的编码，注意文件名有修正
database["felix"]       = img_to_encoding("images/felix.jpg", FRmodel)     # Felix 的编码
database["benoit"]      = img_to_encoding("images/benoit.jpg", FRmodel)    # Benoit 的编码
database["arnaud"]      = img_to_encoding("images/arnaud.jpg", FRmodel)    # Arnaud 的编码

# 打印提示信息，显示数据库创建完成，并统计已添加的授权人员数量
print("✓ 数据库创建完成！共", len(database), "个授权人员")
# len(database) : 返回字典中键值对的数量，即授权人员总数


✓ 数据库创建完成！共 12 个授权人员


现在，当有人来到你家门口刷身份证（提供姓名）时，你可以在数据库中查找他们的编码，并用它来验证站在门前的人是否与身份证上的姓名一致。

**说明**：实现 `verify()` 函数，该函数检查前门摄像头拍摄的图片 (`image_path`) 是否确实是名为 `"identity"` 的人。你需要完成以下步骤：

1. 计算 `image_path` 对应图像的人脸编码
2. 计算此编码与数据库中 `"identity"` 对应编码之间的距离
3. 如果距离小于 0.7，则开门；否则不打开门

如上所述，应使用 L2 距离（`np.linalg.norm`）。注意：在此实现中，将 L2 距离本身（而非平方）与阈值 0.7 进行比较。


In [7]:
# ========================================
# 人脸验证函数 (verify)
# ========================================

def verify(image_path, identity, database, model):
    """
    验证指定图像中的人脸是否为给定身份(identity)
    
    参数:
        image_path -- 图像路径 (str)
        identity -- 要验证的身份名称 (str)，必须在数据库中存在
        database -- Python 字典，键为姓名，值为对应的人脸编码(512维向量)
        model -- 已加载的 InceptionResnetV1 模型，用于提取人脸特征
        
    返回:
        dist -- 与数据库中该身份人脸编码的距离 (float)
        door_open -- 是否开门 (bool)，True表示身份匹配，可开门，False表示不匹配
    """
    
    ### -----------------------------
    ### Step 1: 计算输入图像的编码
    ### -----------------------------
    # 使用之前定义的 img_to_encoding() 函数
    # encoding : 512维人脸特征向量
    encoding = img_to_encoding(image_path, model)
    
    ### -----------------------------
    ### Step 2: 计算与数据库中该身份编码的距离
    ### -----------------------------
    # np.linalg.norm(x) : 计算向量x的L2范数（欧氏距离）
    # encoding - database[identity] : 计算输入图像编码与目标身份编码的差
    # dist : 标量，表示两个编码之间的欧氏距离
    dist = np.linalg.norm(encoding - database[identity])
    
    ### -----------------------------
    ### Step 3: 判断是否开门
    ### -----------------------------
    # 如果距离小于阈值0.7，认为是同一人
    if dist < 0.7:
        print("这是" + str(identity) + ", 你好!")  # 输出欢迎信息
        door_open = True  # 开门
    else:
        print("It's not " + str(identity) + ", 请离开")  # 输出拒绝信息
        door_open = False  # 不开门
    
    # 返回距离和开门状态
    return dist, door_open


Younes 正在尝试进入快乐屋，摄像头拍下了他的照片 ("images/camera_0.jpg")。让我们用你的验证算法来检查这张照片：

<img src="images/camera_0.jpg" style="width:100px;height:100px;">


In [8]:
# ========================================
# 调用 verify 函数进行人脸验证
# ========================================

# verify() 函数作用：
# - 输入图像路径和待验证身份
# - 与数据库中对应身份编码进行比对
# - 输出欧氏距离和是否开门的布尔值

# 参数说明：
# "images/camera_0.jpg" : 待验证的图像路径，通常是摄像头拍摄的照片
# "younes"              : 需要验证的身份名称，必须在 database 中存在
# database              : 事先创建的人脸编码数据库 (字典)
# FRmodel               : 已加载的 InceptionResnetV1 模型，用于提取人脸特征

# 返回值：
# dist      : 输入图像与目标身份编码的欧氏距离 (float)
# door_open : 是否开门 (bool)，True表示身份匹配，False表示不匹配
verify("images/camera_0.jpg", "younes", database, FRmodel)


这是younes, 你好!


(0.5359184, True)

**Expected Output**:

<table>
    <tr>
        <td>
            **输出说明**
        </td>
        <td>
           由于使用随机初始化权重，输出的距离值会不同。<br>
           在使用预训练权重时，会得到类似 (0.659, True) 的结果。
        </td>
    </tr>

</table>

注意：要获得准确的人脸识别结果，需要加载预训练的模型权重。

上周末打破水族箱的 Benoit 已被禁止进入房子，并从数据库中移除。他偷了 Kian 的身份证，并试图回到房子里冒充 Kian。前门摄像头拍摄了 Benoit 的照片（"images/camera_2.jpg"）。让我们运行验证算法，检查 Benoit 是否可以进入。

<img src="images/camera_2.jpg" style="width:100px;height:100px;">


In [9]:
# ========================================
# 调用 verify 函数验证另一张图像
# ========================================

# verify() 函数作用：
# - 检查输入图像中人脸是否与指定身份匹配
# - 输出与目标身份编码的距离和开门状态

# 参数说明：
# "images/camera_2.jpg" : 待验证的图像路径，这里模拟摄像头拍摄的照片
# "kian"                : 要验证的身份名称，必须在数据库 database 中存在
# database              : 事先创建的人脸编码数据库 (字典)，存储所有授权人员编码
# FRmodel               : 已加载的 InceptionResnetV1 模型，用于提取人脸特征向量

# 返回值：
# dist      : 输入图像与目标身份编码的欧氏距离 (float)
# door_open : 是否开门 (bool)，True表示身份匹配，False表示不匹配

verify("images/camera_2.jpg", "kian", database, FRmodel)


It's not kian, 请离开


(1.2924544, False)

**Expected Output**:

<table>
    <tr>
        <td>
            **输出说明**
        </td>
        <td>
           由于使用随机初始化权重，输出的距离值会不同。<br>
           在使用预训练权重时，会得到类似 (0.862, False) 的结果。
        </td>
    </tr>

</table>

### 4.2 - 面部识别

你的面部验证系统基本工作良好。但由于 Kian 的身份证被盗，当他当天晚上回到房子时，无法进入！  

为了减少这种情况，你希望将面部验证系统改为面部识别系统。这样就不需要携带身份证了。授权人员可以直接走到房子前，前门就会为他们解锁！  

你将实现一个面部识别系统，该系统以一张图片作为输入，并判断该图片是否属于授权人员（如果是的话，还能识别是谁）。与之前的面部验证系统不同，我们不再把人的名字作为另一个输入。  

**说明**: 实现 `who_is_it()` 函数。你需要完成以下步骤：
1. 计算来自 `image_path` 的目标编码（target encoding）。
2. 在数据库中找到与目标编码距离最小的编码：
    - 将 `min_dist` 变量初始化为一个足够大的数（如 100）。这将帮助你追踪与输入编码最接近的数据库编码。
    - 遍历数据库字典中的姓名和编码。使用 `for (name, db_enc) in database.items()` 来遍历。
        - 计算目标编码 `encoding` 与当前数据库编码 `db_enc` 之间的 L2 距离。
        - 如果该距离小于 `min_dist`，则更新 `min_dist = dist`，并将 `identity = name`。


In [13]:
# ========================================
# 人脸识别函数 (who_is_it)
# ========================================

def who_is_it(image_path, database, model):
    """
    实现人脸识别：找到输入图像(image_path)对应的人是谁
    
    参数:
        image_path -- 待识别图像路径 (str)
        database -- 人脸编码数据库 (字典)，键为姓名，值为对应的512维特征向量
        model -- 已加载的 InceptionResnetV1 模型，用于提取人脸特征
        
    返回:
        min_dist -- 输入图像编码与数据库中最接近编码的欧氏距离 (float)
        identity -- 识别出的姓名 (str)
    """
    
    ### -----------------------------
    ### Step 1: 计算输入图像的编码
    ### -----------------------------
    # 使用 img_to_encoding() 将输入图像转换为 512 维人脸特征向量
    encoding = img_to_encoding(image_path, model)
    
    ### -----------------------------
    ### Step 2: 在数据库中寻找最接近的编码
    ### -----------------------------
    # 初始化最小距离为一个较大值 (例如100)
    min_dist = 100
    
    # 遍历数据库中的每一个姓名及其对应的编码
    for (name, db_enc) in database.items():
        
        # 计算输入图像编码与数据库中当前编码的欧氏距离
        # np.linalg.norm(x) : 计算向量x的L2范数
        dist = np.linalg.norm(encoding - db_enc)
        
        # 如果当前距离小于记录的最小距离，则更新最小距离和识别身份
        if dist < min_dist:
            min_dist = dist  # 更新最小距离
            identity = name  # 更新当前最可能的身份
    
    ### -----------------------------
    ### Step 3: 输出识别结果
    ### -----------------------------
    # 如果最小距离大于阈值 0.7，则认为数据库中没有该人
    if min_dist > 0.7:
        print("不在数据库中.")
    else:
        # 否则输出识别出的姓名及距离
        print("这是 " + str(identity) + ", 距离是 " + str(min_dist))
        
    # 返回最小距离和识别结果
    return min_dist, identity


In [17]:
# ========================================
# 编码向量分析函数 (analyze_encodings)
# ========================================

def analyze_encodings(database, model, test_image_path):
    """
    分析指定图像的编码向量与数据库编码的距离分布情况
    
    参数:
        database -- 人脸编码数据库 (字典)，键为姓名，值为对应512维特征向量
        model -- 已加载的 InceptionResnetV1 模型，用于提取人脸特征
        test_image_path -- 待分析的图像路径 (str)
    
    返回:
        最接近的姓名 (str)
        对应的最小距离 (float)
    """
    
    # 打印分割线，美化输出
    print("=" * 70)
    print("编码向量分析")
    print("=" * 70)
    
    # Step 1: 获取测试图像的编码
    test_encoding = img_to_encoding(test_image_path, model)  # 512维特征向量
    
    # Step 2: 打印测试图像编码的统计信息
    print(f"\n1. 测试图像编码统计:")
    print(f"   形状: {test_encoding.shape}")                # (1, 512)
    print(f"   范数: {np.linalg.norm(test_encoding):.6f}")  # L2 范数
    print(f"   均值: {np.mean(test_encoding):.6f}")        # 均值
    print(f"   标准差: {np.std(test_encoding):.6f}")       # 标准差
    print(f"   最小值: {np.min(test_encoding):.6f}")       # 最小值
    print(f"   最大值: {np.max(test_encoding):.6f}")       # 最大值
    
    # Step 3: 打印数据库中各人员编码的范数
    print(f"\n2. 数据库中各人员编码的范数:")
    for name, encoding in database.items():
        norm = np.linalg.norm(encoding)
        print(f"   {name:12s}: {norm:.6f}")  # {:12s} 用于格式化对齐姓名
    
    # Step 4: 计算测试图像与数据库各人员的距离
    print(f"\n3. 测试图像与数据库各人员的距离:")
    distances = {}  # 存储姓名: 距离
    for name, db_encoding in database.items():
        dist = np.linalg.norm(test_encoding - db_encoding)  # 欧氏距离
        distances[name] = dist
        print(f"   {name:12s}: {dist:.6f}")
    
    # Step 5: 按距离升序排序，找出最接近的人员
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    
    # Step 6: 打印距离统计信息
    print(f"\n4. 距离统计:")
    dist_values = list(distances.values())
    print(f"   最小距离: {min(dist_values):.6f} ({sorted_distances[0][0]})")
    print(f"   最大距离: {max(dist_values):.6f} ({sorted_distances[-1][0]})")
    print(f"   平均距离: {np.mean(dist_values):.6f}")
    print(f"   标准差: {np.std(dist_values):.6f}")
    
    # Step 7: 打印最接近的前5个人
    print(f"\n5. 最接近的5个人:")
    for i, (name, dist) in enumerate(sorted_distances[:5]):
        print(f"   {i+1}. {name:12s}: {dist:.6f}")
    
    # Step 8: 编码向量相似度参考分析
    print(f"\n6. 编码向量相似度分析:")
    db_names = list(database.keys())
    if len(db_names) >= 2:
        # 随机选取数据库中前两个人的编码距离作为参考
        dist_ref = np.linalg.norm(database[db_names[0]] - database[db_names[1]])
        print(f"   参考：{db_names[0]} vs {db_names[1]} 的距离: {dist_ref:.6f}")
   
   
    # 返回最接近的人员姓名及距离
    return sorted_distances[0][0], sorted_distances[0][1]



Younes 站在前门，摄像头拍下了他的照片（"images/camera_0.jpg"）。让我们看看你的 `who_is_it()` 算法是否能识别出 Younes。


In [18]:
who_is_it("images/camera_0.jpg", database, FRmodel)

这是 younes, 距离是 0.5359184


(0.5359184, 'younes')

**Expected Output**:

<table>
    <tr>
        <td>
            **输出说明**
        </td>
        <td>
           由于使用随机初始化权重，输出的距离值会不同。<br>
           在使用预训练权重时，会得到类似 (0.659, 'younes') 的结果。
        </td>
    </tr>

</table>

In [19]:
# ========================================
# 运行分析函数示例
# ========================================
print("分析 camera_0.jpg 的编码情况...\n")

# predicted_name : 最接近的人员姓名
# min_dist       : 对应的欧氏距离
predicted_name, min_dist = analyze_encodings(database, FRmodel, "images/camera_0.jpg")

分析 camera_0.jpg 的编码情况...

编码向量分析

1. 测试图像编码统计:
   形状: (1, 512)
   范数: 1.000000
   均值: -0.002954
   标准差: 0.044095
   最小值: -0.114437
   最大值: 0.127814

2. 数据库中各人员编码的范数:
   danielle    : 1.000000
   younes      : 1.000000
   tian        : 1.000000
   andrew      : 1.000000
   kian        : 1.000000
   dan         : 1.000000
   sebastiano  : 1.000000
   bertrand    : 1.000000
   kevin       : 1.000000
   felix       : 1.000000
   benoit      : 1.000000
   arnaud      : 1.000000

3. 测试图像与数据库各人员的距离:
   danielle    : 1.275298
   younes      : 0.535918
   tian        : 1.281834
   andrew      : 1.565095
   kian        : 1.334232
   dan         : 1.296595
   sebastiano  : 1.504536
   bertrand    : 1.354992
   kevin       : 1.314591
   felix       : 1.437835
   benoit      : 1.294205
   arnaud      : 1.179268

4. 距离统计:
   最小距离: 0.535918 (younes)
   最大距离: 1.565095 (andrew)
   平均距离: 1.281200
   标准差: 0.246755

5. 最接近的5个人:
   1. younes      : 0.535918
   2. arnaud      : 1.179268
   3. danielle    : 

你的快乐之家运行良好。它只允许授权人员进入，人们也不需要随身携带身份证了！

你现在已经了解了一个最先进的人脸识别系统是如何工作的。

虽然我们在这里不会实现这些改进，但以下是一些可以进一步提高算法性能的方法：
- 为每个人在数据库中加入更多图片（在不同光照条件下拍摄、不同日期拍摄等）。然后在给定新图像时，将新面孔与该人的多张图片进行对比。这将提高识别准确性。
- 将图像裁剪为仅包含面部，而减少面部周围的“边界”区域。这种预处理可以去掉一些无关的像素，同时也让算法更稳健。


<font color='blue'>
    
**你需要记住的内容**：
- 人脸验证（Face Verification）解决的是较简单的 1:1 匹配问题；人脸识别（Face Recognition）则是较复杂的 1:K 匹配问题。
- 三元组损失（Triplet Loss）是训练神经网络以学习面部图像编码的有效损失函数。
- 相同的编码可以用于验证和识别。通过测量两张图像编码之间的距离，可以判断它们是否属于同一个人。


### 参考文献：

- Florian Schroff, Dmitry Kalenichenko, James Philbin (2015). [FaceNet: A Unified Embedding for Face Recognition and Clustering](https://arxiv.org/pdf/1503.03832.pdf)
- Yaniv Taigman, Ming Yang, Marc'Aurelio Ranzato, Lior Wolf (2014). [DeepFace: Closing the gap to human-level performance in face verification](https://research.fb.com/wp-content/uploads/2016/11/deepface-closing-the-gap-to-human-level-performance-in-face-verification.pdf) 
- 我们使用的预训练模型 使用了facenet_pytorch 提供的 Inception-ResNet v1 架构
- 我们的实现也参考了官方 FaceNet GitHub 仓库的许多内容：https://github.com/davidsandberg/facenet
